## Wordle

In [112]:
import pandas as pd
import random
import numpy as np
from docplex.cp.model import CpoModel

In [113]:
def score(guess, todays_word):
    score = ['?'] * 5  # X represents gray
    todays_word = list(todays_word)
    
    for i in range(5):  # Green letters first ($)
        if guess[i] == todays_word[i]:
            todays_word[i] = '$'
            score[i] = '$'
            
    for i in range(5):  # Yellow letters (%)
        if score[i] != '$' and guess[i] in todays_word:
            score[i] = '%'
            replace_index = ''.join(todays_word).find(guess[i])
            todays_word[replace_index] = '%'  # This should not be i but whatever index matches
            
    return score

In [120]:
def cp_model(valid_words, guess_outcomes):
    if not guess_outcomes:  # Initial Guess 
        return 'arose'
    
    num_letters = 26
    num_pos = 5
    num_guesses = len(guess_outcomes)
    
    guess_hist = [[ord(letter) - 97 for letter in list(prev[0])] for prev in guess_outcomes]
    
    num_correct = [prev[1].count('$') for prev in guess_outcomes]
    num_present = [prev[1].count('%') for prev in guess_outcomes]
    
    m = CpoModel()
    
    # Decision Variables
    w = m.integer_var_list(num_pos, min=0, max=num_letters-1, name='w')  # Guess
    numberSolution = m.integer_var_list(num_letters,min=0,max=4, name="numberSolution")  # Max is 4 bc no 5 letter word is all 1 character (probably actually 3)
    numberLetterPresent = m.integer_var_dict(((g,l) for g in range(num_guesses) for l in range(num_letters)),min=0,max=num_pos,name="numberLetterPresent")
    numberLetterRight = m.integer_var_dict(((g,l) for g in range(num_guesses) for l in range(num_letters)),min=0,max=num_pos,name="numberLetterRight")
    # test = m.integer_var_list(num_pos,min=0,max=num_letters-1,name="test")
    
    # Constraints
    # Guesses are in the word list and not already guessed
    m.add(m.allowed_assignments(w, [[ord(letter) - 97 for letter in list(word)] for word in valid_words]))
    m.add(m.forbidden_assignments(w, guess_hist))
    
    for p in range(num_pos):
        if guess_outcomes[-1][1][p] == '$':
            m.add(w[p] == guess_hist[-1][p])
        elif guess_outcomes[-1][1][p] == '%':  # This needs to be 0 + right, maybe make a num right dvar???
            m.add(numberSolution[guess_hist[-1][p]] > 0)
        else:  # This needs to be for every previous guess
            m.add(numberSolution[guess_hist[-1][p]] == 0)  #Change to num already right
    
    for guess in range(num_guesses):
        # Correct number of correct letters
        m.add(m.sum((w[p] == guess_hist[guess][p]) * 1 for p in range(5)) == num_correct[guess])
        # Correct number of present letters
        m.add(m.sum((numberLetterPresent[guess,letter] >= 1) for letter in range(num_letters)) == num_present[guess])
    
    for letter in range(num_letters):
        # Number of times a letter appears in the solution
        m.add(numberSolution[letter] == m.sum(w[p] == letter for p in range(5)))
    
    for guess in range(num_guesses):
        for letter in range(num_letters):
            # Number of times a letter in the guess is present
            m.add(numberLetterPresent[guess,letter] == m.sum(((guess_hist[guess][p] == letter) & (w[p] != letter) & (numberSolution[letter] > numberLetterRight[guess, letter])) * 1 for p in range(num_pos)))
            # Number of times a letter in the guess is correct
            m.add(numberLetterRight[guess,letter] == m.sum(((guess_hist[guess][p] == letter) & (w[p] == letter)) * 1 for p in range(num_pos)))
  

    # Solve
    mGuess = m.solve()  #m.solve(LogVerbosity='Quiet')
    mGuessStr = ""
    num = mGuess.get_all_var_solutions()
    
    for i in range(num_pos):
        mGuessStr += chr(num[i].get_value() + 97)
    return mGuessStr

In [121]:
def play(manual_score=False):
    valid_words = list(set(open('Data/wordle_valid_guesses.txt').read().lower().splitlines()))
    
    if not manual_score:
        todays_word = random.choice(valid_words)
        print(todays_word)
        guess_outcomes = []
        n=0
        while True:
            guess = cp_model(valid_words, guess_outcomes)
            print(guess)
            output = score(guess, todays_word)
            print(output)
            if output == list('$$$$$'):  # $ - green, ? - gray, % - yellow
                break
            guess_outcomes.append((guess, output))
            n += 1
            
    return n

In [122]:
play()

tease
arose
['%', '?', '?', '$', '$']
 ! --------------------------------------------------- CP Optimizer 22.1.0.0 --
 ! Satisfiability problem - 83 variables, 87 constraints
 ! Presolve      : 24 extractables eliminated, 1 constraint generated
 ! Initial process time : 0.02s (0.02s extraction + 0.00s propagation)
 !  . Log search space  : 38.6 (before), 38.6 (after)
 !  . Memory usage      : 367.6 kB (before), 367.6 kB (after)
 ! Using parallel search with 8 workers.
 ! ----------------------------------------------------------------------------
 !               Branches  Non-fixed    W       Branch decision
 *                      2  0.03s        1        10  = w_1
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! ----------------------------------------------------------------------------
 ! Number of branches     : 44
 ! Number of fails        : 13
 ! Total memory usage     : 12.0 MB (11.9 MB CP Optimizer + 0.0

TypeError: 'NoneType' object is not subscriptable